In [1]:
import numpy as np
import pandas as pd
import os
import keras 
from keras.models import Sequential 
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from sklearn.metrics import accuracy_score
from ipywidgets import widgets
import io
from PIL import Image
import tqdm
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
import tensorflow as tf

In [2]:
path = os.listdir('brain_tumor/Training/')
labels = ['no_tumor',  'pituitary_tumor', 'glioma_tumor', 'meningioma_tumor']

In [31]:
X_train = []
Y_train = []
for cls in labels:
    pth = 'brain_tumor/Training/'+cls
    for j in os.listdir(pth):
        img = cv2.imread(pth+'/'+j)
        img = cv2.resize(img, (80,80))
        X_train.append(img)
        Y_train.append(cls)
for cls in labels:
    pth = 'brain_tumor/Testing/'+cls
    for j in os.listdir(pth):
        img = cv2.imread(pth+'/'+j)
        img = cv2.resize(img, (80,80))
        X_train.append(img)
        Y_train.append(cls)

In [32]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [24]:
X_train.shape

(3264, 80, 80, 3)

In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [33]:
datagen = ImageDataGenerator(
    rotation_range=90,horizontal_flip=False,vertical_flip=True)
X_train_new = X_train
for i in range (2):
    datagen.fit(X_train_new)
    X_train = np.concatenate((X_train,X_train_new))

In [34]:
Y_train_new = Y_train
Y_train = np.concatenate((Y_train,Y_train_new))
Y_train = np.concatenate((Y_train,Y_train_new))

In [35]:
Y_train.shape

(9792,)

In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 101)

In [37]:
Y_train_new = []
for i in Y_train:
    Y_train_new.append(labels.index(i))
    
Y_train = Y_train_new
Y_train = tf.keras.utils.to_categorical(Y_train)

Y_test_new = []
for i in Y_test:
    Y_test_new.append(labels.index(i))
    
Y_test = Y_test_new
Y_test = tf.keras.utils.to_categorical(Y_test)

In [38]:
Y_train

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [39]:
model = Sequential()
model.add(Conv2D(64,(2,2),activation = 'relu',input_shape=(80,80,3)))
model.add(Conv2D(64,(2,2),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.1))
model.add(Conv2D(32,(2,2),activation='relu'))
model.add(Conv2D(32,(2,2),activation='relu'))
#model.add(Dropout(0.3))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.1))
model.add(Conv2D(16,(2,2),activation='relu'))
model.add(Conv2D(16,(2,2),activation='relu'))
#model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.1))
model.add(Conv2D(8,(2,2),activation='relu'))
model.add(Conv2D(8,(2,2),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(1024,activation = 'relu'))
#model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(4,activation='softmax'))

In [40]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [43]:
history = model.fit(X_train, Y_train, epochs = 30, validation_split = 0.1)

Epoch 1/30
248/248 [==============================] - 112s 453ms/step - loss: 0.1943 - accuracy: 0.9265 - val_loss: 0.1543 - val_accuracy: 0.9422
Epoch 2/30
248/248 [==============================] - 131s 528ms/step - loss: 0.1955 - accuracy: 0.9267 - val_loss: 0.1262 - val_accuracy: 0.9535
Epoch 3/30
248/248 [==============================] - 104s 419ms/step - loss: 0.1691 - accuracy: 0.9366 - val_loss: 0.0865 - val_accuracy: 0.9739
Epoch 4/30
248/248 [==============================] - 141s 570ms/step - loss: 0.1660 - accuracy: 0.9377 - val_loss: 0.1359 - val_accuracy: 0.9580
Epoch 5/30
248/248 [==============================] - 100s 401ms/step - loss: 0.1649 - accuracy: 0.9391 - val_loss: 0.0756 - val_accuracy: 0.9785
Epoch 6/30
248/248 [==============================] - 96s 387ms/step - loss: 0.1463 - accuracy: 0.9469 - val_loss: 0.0583 - val_accuracy: 0.9875
Epoch 7/30
248/248 [==============================] - 97s 390ms/step - loss: 0.1498 - accuracy: 0.9487 - val_loss: 0.0777 - v

In [45]:
loss, accuracy = model.evaluate(X_train, Y_train)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

276/276 [==============================] - 15s 55ms/step - loss: 0.0142 - accuracy: 0.9965
Test loss: 0.014240228570997715
Test accuracy: 0.9964820742607117
